In [1]:
!pip install \
  langchain_community \
  langchain_pinecone \
  langchain_openai \
  unstructured \
  langchain-text-splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.2/361.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 24.2 MB/s eta 0:00:00
   

In [7]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import glob

In [12]:
text_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader("./full_data/", glob="./*.txt", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)

In [13]:
docstest = loader.load()

In [23]:
from google.colab import userdata

os.environ['OPENAPI_KEY_API'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [24]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)
index_name = "pinecone-test"

#split our documents into chunks
text_splitter = RecursiveCharacterTextSplitter()
split_docs = text_splitter.split_documents(docstest)

In [27]:
vectorstore = PineconeVectorStore.from_documents(split_docs, embeddings, index_name=index_name)

In [44]:
query = "What is Saanvi's last name?"
similar_docs = vectorstore.similarity_search(query)

In [41]:
similar_docs[0]

Document(metadata={'source': 'full_data/3040PaperSaanviBhumpalle.txt'}, page_content='Deep learning in computer vision provides many levels of abstraction that mimic the deep neural networks in our brains. Deep neural networks in our brains are extremely “complex” and still not completely understood (Kriegeskorte, 2015). By contrast, computational models have artificial neural networks that have well-defined layers with matrix multiplication, stochastic pooling, convolution, and other mathematical operations. Humans are better at context and understanding the meaning behind a visual scene, unlike computers. Perhaps artificial neural networks can incorporate layers for deriving information from the context of a scene through semantical analysis. Humans all have different biases when gathering the gist from a visual scene. Still, computer vision would gather unbiased input that computer scientists would achieve by cleaning data sets and removing bias from those (Kyle-Davidson et al., 202

In [45]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

qa.invoke(query)

{'query': "What is Saanvi's last name?",
 'result': "Saanvi's last name is Bhumpalle."}